In [ ]:
conda activate lenci_enviroment


In [26]:
import pandas as pd
import surprise
from surprise import SVD, Reader
from recommenders.models.surprise.surprise_utils import predict, compute_ranking_predictions
from recommenders.datasets.python_splitters import python_random_split
from recommenders.evaluation.python_evaluation import (
    rmse, mae, rsquared, exp_var, map_at_k, ndcg_at_k, precision_at_k, recall_at_k
)
from collections import defaultdict

In [27]:
# =================== 1. Preparar dataset ===================

file_path = "C:\\Users\\vlenc\\OneDrive\\Documentos\\ml-latest-small\\ratings.csv"
movies_file = "C:\\Users\\vlenc\\OneDrive\\Documentos\\ml-latest-small\\movies.csv"

df = pd.read_csv(file_path)
df = df[["userId", "movieId", "rating"]]
# Padronizar nomes de colunas para Recommenders
df = df.rename(columns={"userId": "userID", "movieId": "itemID", "rating": "rating"})

movies = pd.read_csv(movies_file)
movies = movies.rename(columns={"movieId": "itemID", "title": "title"})
# movies = movies.merge(df.groupby("itemID").size().reset_index(name="num_ratings"), on="itemID")
# movies = movies.sort_values("num_ratings", ascending=False).reset_index(drop=True)

In [28]:
# =================== 2. Selecionar usuário holdout ===================

# --- Selecionar um usuário para teste de fold-in (será removido do treinamento) ---
user_counts = df.groupby("userID").size().reset_index(name="num_ratings")
user_counts = user_counts[user_counts["num_ratings"] >= 20].sort_values("num_ratings", ascending=False)

print("Usuários com mais avaliações (candidatos para holdout):")
print(user_counts.head(10))
print()

# Você pode escolher manualmente ou pegar automaticamente
HOLDOUT_USER_ID = int(user_counts.iloc[10]["userID"])  # Pega o 6º usuário com mais avaliações
# Ou defina manualmente: HOLDOUT_USER_ID = 4

print(f"✓ Usuário selecionado para holdout (fold-in): {HOLDOUT_USER_ID}")
print(f"  Este usuário NÃO participará do treinamento.\n")

# Separar dados do usuário holdout
user_holdout_data = df[df["userID"] == HOLDOUT_USER_ID].copy()
df_sem_holdout = df[df["userID"] != HOLDOUT_USER_ID].copy()

print(f"Dataset original: {len(df)} avaliações")
print(f"Avaliações do usuário holdout: {len(user_holdout_data)}")
print(f"Dataset para treinamento (sem holdout): {len(df_sem_holdout)} avaliações\n")

Usuários com mais avaliações (candidatos para holdout):
     userID  num_ratings
413     414         2698
598     599         2478
473     474         2108
447     448         1864
273     274         1346
609     610         1302
67       68         1260
379     380         1218
605     606         1115
287     288         1055

✓ Usuário selecionado para holdout (fold-in): 249
  Este usuário NÃO participará do treinamento.

Dataset original: 100836 avaliações
Avaliações do usuário holdout: 1046
Dataset para treinamento (sem holdout): 99790 avaliações



In [29]:
# =================== 4. Criar treino/teste (sem o usuário holdout)===================

# A seed é o que faz a base de treino/teste sempre ser a mesma.
train_df, test_df = python_random_split(df_sem_holdout, 0.8, seed=42)

print(f"Train set: {len(train_df)} avaliações")
print(f"Test set: {len(test_df)} avaliações\n")

# Criar dataset do Surprise
reader = Reader(rating_scale=(1, 5))

# Adaptar DataFrame 'train_df' para ser utilizado na função 'SVD'
trainset = surprise.Dataset.load_from_df(train_df, reader).build_full_trainset()

Train set: 79832 avaliações
Test set: 19958 avaliações



In [30]:
# =================== 5. Treinar SVD ===================
svd = SVD(n_factors=200, n_epochs=30, random_state=42, verbose=True)
svd.fit(trainset)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing epoch 24
Processing epoch 25
Processing epoch 26
Processing epoch 27
Processing epoch 28
Processing epoch 29


In [31]:
# =================== 6. Criar predições ===================
predictions = predict(svd, test_df, usercol="userID", itemcol="itemID")
predictions.head(5)

,userID,itemID,prediction
0,68,923,3.167365
1,434,292,3.434771
2,396,783,3.177512
3,399,2959,4.057186
4,500,543,2.871088


In [32]:
# =================== 7. Criar predições com o treinamento para comparação ===============

# É utilizado para avaliar as métricas de ranking (MAP, NDCG, Precision@K, Recall@K).

all_predictions = compute_ranking_predictions(
    svd, train_df, usercol="userID", itemcol="itemID", remove_seen=True
)
all_predictions.head()

# No bloco de código seguinte é feito uma comparação entre as predições e as notas reais

,userID,itemID,prediction
1,1,2,4.578591
3,1,4,3.953589
4,1,5,3.544043
6,1,7,4.194905
7,1,8,3.911575


In [33]:
# =================== 7. Avaliação das predições ====================

# “O modelo acerta a nota que o usuário deu para o filme?”
eval_rmse = rmse(test_df, predictions)
eval_mae = mae(test_df, predictions)
eval_rsquared = rsquared(test_df, predictions)
eval_exp_var = exp_var(test_df, predictions)

# O modelo está colocando os filmes certos (que o usuário realmente gosta) nas primeiras posições das recomendações?
eval_map = map_at_k(test_df, all_predictions, col_prediction="prediction", k=10)
eval_ndcg = ndcg_at_k(test_df, all_predictions, col_prediction="prediction", k=10)
eval_precision = precision_at_k(
    test_df, all_predictions, col_prediction="prediction", k=10
)
eval_recall = recall_at_k(test_df, all_predictions, col_prediction="prediction", k=10)


print(
    "RMSE:\t\t%f" % eval_rmse,
    "MAE:\t\t%f" % eval_mae,
    "rsquared:\t%f" % eval_rsquared,
    "exp var:\t%f" % eval_exp_var,
    sep="\n",
)

print("----")

print(
    "MAP:\t\t%f" % eval_map,
    "NDCG:\t\t%f" % eval_ndcg,
    "Precision@K:\t%f" % eval_precision,
    "Recall@K:\t%f" % eval_recall,
    sep="\n",
)

RMSE:		0.882922
MAE:		0.678920
rsquared:	0.286944
exp var:	0.287220
----
MAP:		0.028756
NDCG:		0.065049
Precision@K:	0.054516
Recall@K:	0.021710


In [34]:
# =================== 8. Fold-in com usuário holdout (cold-start real) ===================
import numpy as np

MIN_AVALIACOES_PARA_FOLDIN = 5
LAMBDA_REG = 0.1
TOP_K = 10
SPLIT_RATIO = 0.8  # 80% para fold-in, 20% para validação

print(f"🎬 TESTE DE FOLD-IN COM USUÁRIO REAL (COLD-START)")

print(f"Usuário holdout: {HOLDOUT_USER_ID}")
print(f"Este usuário NÃO participou do treinamento do modelo.\n")

# Verificar se temos dados do usuário
if len(user_holdout_data) == 0:
    raise RuntimeError(f"Nenhuma avaliação encontrada para o usuário {HOLDOUT_USER_ID}")

print(f"Total de avaliações do usuário: {len(user_holdout_data)}\n")

# --- PASSO 2: Dividir avaliações em conhecidas e ocultas ---
user_ratings_shuffled = user_holdout_data.sample(frac=1, random_state=42).reset_index(drop=True)
split_point = int(len(user_ratings_shuffled) * SPLIT_RATIO)

known_ratings = user_ratings_shuffled.iloc[:split_point].copy()
hidden_ratings = user_ratings_shuffled.iloc[split_point:].copy()

print(f"📊 Divisão das avaliações:")
print(f"  Avaliações conhecidas (para fold-in): {len(known_ratings)}")
print(f"  Avaliações ocultas (para validação): {len(hidden_ratings)}\n")

# Mostrar algumas avaliações conhecidas
print("Exemplos de filmes que o usuário avaliou (conhecidas):")
known_with_titles = known_ratings.merge(movies[["itemID", "title", "genres"]], on="itemID", how="left")
for idx, row in known_with_titles.head(5).iterrows():
    first_genre = row["genres"].split("|")[0] if isinstance(row.get("genres", None), str) else "Desconhecido"
    print(f"  • {row['title']} (Gênero: {first_genre}) → Nota: {row['rating']}")
print()

# --- PASSO 3: Filtrar apenas filmes que estão no trainset ---
train_raw_ids = [trainset.to_raw_iid(i) for i in range(trainset.n_items)]
known_ratings_in_train = known_ratings[known_ratings["itemID"].isin(train_raw_ids)].copy()

print(f"Avaliações conhecidas disponíveis no trainset: {len(known_ratings_in_train)}/{len(known_ratings)}")

if len(known_ratings_in_train) < MIN_AVALIACOES_PARA_FOLDIN:
    print(f"\n⚠️ ERRO: Apenas {len(known_ratings_in_train)} avaliações válidas.")
    print(f"Necessário pelo menos {MIN_AVALIACOES_PARA_FOLDIN} para fold-in.")
    print("Sugestões:")
    print("  - Escolha um usuário com mais avaliações")
    print("  - Reduza MIN_AVALIACOES_PARA_FOLDIN")
    print("  - Aumente SPLIT_RATIO para ter mais avaliações conhecidas\n")
else:
    print(f"✓ Suficiente para fold-in!\n")
    
    # --- PASSO 4: Aplicar fold-in com avaliações conhecidas ---
    avaliacoes = [(int(row["itemID"]), float(row["rating"])) 
                  for _, row in known_ratings_in_train.iterrows()]
    
    # Mapear avaliações para inner ids
    inner_ids = []
    ratings = []
    erros = 0
    for raw_iid, r in avaliacoes:
        try:
            inner = trainset.to_inner_iid(raw_iid)
            inner_ids.append(inner)
            ratings.append(r)
        except ValueError:
            erros += 1
            continue
    
    print(f"Mapeamento para inner IDs:")
    print(f"  ✓ Sucesso: {len(inner_ids)}")
    if erros > 0:
        print(f"  ✗ Falhas: {erros}")
    print()
    
    # Construir Q e y (resíduos)
    mu = trainset.global_mean
    k = svd.n_factors
    Q = np.array([svd.qi[i] for i in inner_ids])
    bi = np.array([svd.bi[i] for i in inner_ids])
    y = np.array(ratings) - mu - bi
    
    print("Calculando vetor latente do usuário via fold-in...")
    
    # Resolver (Q^T Q + lambda I) p_u = Q^T y
    A = Q.T.dot(Q) + LAMBDA_REG * np.eye(k)
    b = Q.T.dot(y)
    try:
        p_u = np.linalg.solve(A, b)
    except np.linalg.LinAlgError:
        p_u = np.linalg.pinv(A).dot(b)
    
    # Estimar bias do usuário
    residuals = y - Q.dot(p_u)
    b_u = float(np.mean(residuals)) if residuals.size > 0 else 0.0
    
    print(f"✓ Vetor latente calculado (dimensão: {len(p_u)})")
    print(f"✓ Bias do usuário: {b_u:.3f}\n")
    
    # --- PASSO 5: Gerar recomendações ---
    print("Gerando predições para todos os filmes do trainset...")
    preds = []
    for inner_j in range(trainset.n_items):
        qj = svd.qi[inner_j]
        bj = svd.bi[inner_j]
        est = mu + bj + b_u + p_u.dot(qj)
        raw_j = trainset.to_raw_iid(inner_j)
        preds.append((raw_j, est))
    
    # Remover itens que o usuário já avaliou (conhecidas)
    avaliados_raw = set([raw for raw, _ in avaliacoes])
    preds = [p for p in preds if p[0] not in avaliados_raw]
    
    print(f"✓ {len(preds)} filmes candidatos (excluindo já avaliados)\n")
    
    # Ordenar e pegar top K
    preds.sort(key=lambda x: x[1], reverse=True)
    topk = preds[:TOP_K]
    
    # Exibir recomendações
    print(f"{'='*70}")
    print(f"🎬 TOP-{TOP_K} RECOMENDAÇÕES PARA O USUÁRIO {HOLDOUT_USER_ID}")
    print(f"{'='*70}\n")
    
    for i, (raw_j, est) in enumerate(topk, 1):
        row = movies[movies["itemID"] == raw_j]
        if not row.empty:
            title = row.iloc[0]["title"]
            genres = row.iloc[0]["genres"] if "genres" in row.columns else ""
            first_genre = genres.split("|")[0] if isinstance(genres, str) and genres else "Desconhecido"
            print(f"{i:2d}. {title}")
            print(f"    Gênero: {first_genre} | Nota estimada: {est:.2f}\n")
        else:
            print(f"{i:2d}. Item ID {raw_j} | Nota estimada: {est:.2f}\n")
    
    # --- PASSO 6: Avaliar qualidade das recomendações ---
    print(f"{'='*70}")
    print(f"📊 AVALIAÇÃO DA QUALIDADE DAS RECOMENDAÇÕES")
    print(f"{'='*70}\n")
    
    # Filmes recomendados (top K)
    recommended_items = set([raw_j for raw_j, _ in topk])
    
    # Filmes que o usuário realmente gostou nas avaliações ocultas (nota >= 4)
    hidden_liked = hidden_ratings[hidden_ratings["rating"] >= 4.0].copy()
    hidden_liked_set = set(hidden_liked["itemID"].tolist())
    
    print(f"Estatísticas:")
    print(f"  • Filmes recomendados: {len(recommended_items)}")
    print(f"  • Filmes que o usuário gostou nas avaliações ocultas (nota ≥ 4): {len(hidden_liked_set)}\n")
    
    # Verificar quantos filmes recomendados o usuário realmente gostou
    hits = recommended_items.intersection(hidden_liked_set)
    
    if len(hidden_liked_set) > 0:
        precision = len(hits) / len(recommended_items) if len(recommended_items) > 0 else 0
        recall = len(hits) / len(hidden_liked_set)
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        
        print(f"Métricas de Performance:")
        print(f"  ✓ Acertos: {len(hits)} filme(s)")
        print(f"  • Precision@{TOP_K}: {precision:.2%}")
        print(f"  • Recall@{TOP_K}: {recall:.2%}")
        print(f"  • F1-Score@{TOP_K}: {f1:.2%}\n")
        
        if len(hits) > 0:
            print(f"🎯 Filmes recomendados que o usuário REALMENTE gostou:\n")
            for item_id in hits:
                row = movies[movies["itemID"] == item_id]
                if not row.empty:
                    title = row.iloc[0]["title"]
                    actual_rating = hidden_ratings[hidden_ratings["itemID"] == item_id]["rating"].values[0]
                    genres = row.iloc[0]["genres"] if "genres" in row.columns else ""
                    first_genre = genres.split("|")[0] if isinstance(genres, str) and genres else "Desconhecido"
                    print(f"  ✓ {title}")
                    print(f"    Gênero: {first_genre} | Nota real: {actual_rating}\n")
        else:
            print("⚠️ Nenhum dos filmes recomendados está entre os que o usuário gostou (ocultos).\n")
    else:
        print("⚠️ O usuário não tem filmes com nota alta (≥4) nas avaliações ocultas.\n")
    
    # Análise adicional: distribuição de notas nas avaliações ocultas
    print(f"Distribuição de notas nas avaliações ocultas:")
    dist = hidden_ratings["rating"].value_counts().sort_index()
    for nota, count in dist.items():
        print(f"  Nota {nota}: {count} filme(s)")
    
    print(f"\n{'='*70}")
    print("✓ Fold-in concluído!")
    print(f"{'='*70}\n")

🎬 TESTE DE FOLD-IN COM USUÁRIO REAL (COLD-START)
Usuário holdout: 249
Este usuário NÃO participou do treinamento do modelo.

Total de avaliações do usuário: 1046

📊 Divisão das avaliações:
  Avaliações conhecidas (para fold-in): 836
  Avaliações ocultas (para validação): 210

Exemplos de filmes que o usuário avaliou (conhecidas):
  • 50/50 (2011) (Gênero: Comedy) → Nota: 3.5
  • A.I. Artificial Intelligence (2001) (Gênero: Adventure) → Nota: 3.5
  • Star Trek (2009) (Gênero: Action) → Nota: 4.0
  • Star Wars: Episode II - Attack of the Clones (2002) (Gênero: Action) → Nota: 3.5
  • Hunt for the Wilderpeople (2016) (Gênero: Adventure) → Nota: 4.5

Avaliações conhecidas disponíveis no trainset: 818/836
✓ Suficiente para fold-in!

Mapeamento para inner IDs:
  ✓ Sucesso: 818

Calculando vetor latente do usuário via fold-in...
✓ Vetor latente calculado (dimensão: 200)
✓ Bias do usuário: 0.088

Gerando predições para todos os filmes do trainset...
✓ 8174 filmes candidatos (excluindo já avali